In [ ]:
#importing files
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy

In [ ]:
#Reading Dataset

data = pd.read_csv("cleaveland.csv" ,header=0)
data['AG']=data['AG'].astype(float)
data['SX']=data['SX'].astype(float)
data['TCP']=data['TCP'].astype(float)
data['RBPR']=data['RBPR'].astype(float)
data['SCL']=data['SCL'].astype(float)
data['FBSR']=data['FBSR'].astype(float)
data['RER']=data['RER'].astype(float)
data['MHRA']=data['MHRA'].astype(float)
data['EXIA']=data['EXIA'].astype(float)
data['STDIE']=data['STDIE'].astype(float)
data['SPE']=data['SPE'].astype(float)
data['NMV']=data['NMV'].astype(float)
data['THM']=data['THM'].astype(float)
data.TAR[data.TAR == 2] = 1
data.TAR[data.TAR == 3] = 1
data.TAR[data.TAR == 4] = 1
data.info()


In [ ]:
#MICE Imputation
from fancyimpute import IterativeImputer


MICE_imputer = IterativeImputer()

data1 = MICE_imputer.fit_transform(data)
data = pd.DataFrame({'AG': data1[:, 0], 'SX': data1[:, 1],'TCP': data1[:, 2],'RBPR': data1[:, 3],'SCL': data1[:, 4],'FBSR': data1[:, 5],'RER': data1[:, 6],'MHRA': data1[:, 7],'EXIA': data1[:, 8],'STDIE': data1[:, 9],'SPE': data1[:, 10],'NMV': data1[:, 11],'THM': data1[:, 12],'TAR': data1[:, 13]})

print(data.isnull().sum())


In [ ]:
#Placing Prediction Variables in x and target variables in y
x=data.drop('TAR',axis=1)
y=data.TAR


In [ ]:
#Select features using Genetic Algorithm
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
 
SEED = 2018
random.seed(SEED)
np.random.seed(SEED)
class GeneticSelector:
    def __init__(self, estimator, n_gen, size, n_best, n_rand, 
                 n_children, mutation_rate):
        # Estimator 
        self.estimator = estimator
        # Number of generations
        self.n_gen = n_gen
        # Number of chromosomes in population
        self.size = size
        # Number of best chromosomes to select
        self.n_best = n_best
        # Number of random chromosomes to select
        self.n_rand = n_rand
        # Number of children created during crossover
        self.n_children = n_children
        # Probablity of chromosome mutation
        self.mutation_rate = mutation_rate
        
        if int((self.n_best + self.n_rand) / 2) * self.n_children != self.size:
            raise ValueError("The population size is not stable.")
    def initilize(self):
        population = []
        for i in range(self.size):
            chromosome = np.ones(self.n_features, dtype=np.bool)
            mask = np.random.rand(len(chromosome)) < 0.3
            chromosome[mask] = False
            population.append(chromosome)
        return population
    def fitness(self, population):
        X, y = self.dataset
        scores = []
        for chromosome in population:
            score = -1.0 * np.mean(cross_val_score(self.estimator, X[:,chromosome], y, 
                                                       cv=5, 
                                                       scoring="neg_mean_squared_error"))
            scores.append(score)
        scores, population = np.array(scores), np.array(population) 
        inds = np.argsort(scores)
        return list(scores[inds]), list(population[inds,:])
    def select(self, population_sorted):
        population_next = []
        for i in range(self.n_best):
            population_next.append(population_sorted[i])
        for i in range(self.n_rand):
            population_next.append(random.choice(population_sorted))
        random.shuffle(population_next)
        return population_next
    def crossover(self, population):
        population_next = []
        for i in range(int(len(population)/2)):
            for j in range(self.n_children):
                chromosome1, chromosome2 = population[i], population[len(population)-1-i]
                child = chromosome1
                mask = np.random.rand(len(child)) > 0.5
                child[mask] = chromosome2[mask]
                population_next.append(child)
        return population_next
    def mutate(self, population):
        population_next = []
        for i in range(len(population)):
            chromosome = population[i]
            if random.random() < self.mutation_rate:
                mask = np.random.rand(len(chromosome)) < 0.05
                chromosome[mask] = False
            population_next.append(chromosome)
        return population_next
    def generate(self, population):
        # Selection, crossover and mutation
        scores_sorted, population_sorted = self.fitness(population)
        population = self.select(population_sorted)
        population = self.crossover(population)
        population = self.mutate(population)
        # History
        self.chromosomes_best.append(population_sorted[0])
        self.scores_best.append(scores_sorted[0])
        self.scores_avg.append(np.mean(scores_sorted))
        
        return population
    def fit(self, X, y):
 
        self.chromosomes_best = []
        self.scores_best, self.scores_avg  = [], []
        
        self.dataset = X, y
        self.n_features = X.shape[1]
        
        population = self.initilize()
        for i in range(self.n_gen):
            population = self.generate(population)
            
        return self 
    
    @property
    def support_(self):
        return self.chromosomes_best[-1]
 
    def plot_scores(self):
        plt.plot(self.scores_best, label='Best')
        plt.plot(self.scores_avg, label='Average')
        plt.legend()
        plt.ylabel('Scores')
        plt.xlabel('Generation')
        plt.show()
sel = GeneticSelector(estimator=LinearRegression(), 
                      n_gen=18, size=200, n_best=40, n_rand=40, 
                      n_children=5, mutation_rate=0.05)

X=x.values
Y=y.values
#X.shape
sel.fit(X, Y)
population=sel.initilize()
population

sel.crossover(population)

print (sel.support_)



In [ ]:
# Prediction variables selected using genetic algorithm 
prediction_var_ga=['SX','TCP','RER','MHRA','EXIA','STDIE','SPE','NMV','THM']

In [ ]:
#finding out optimal number of features using recursive feature elimination

X=data[prediction_var_ga]
Y=data.TAR
from sklearn.feature_selection import RFE

nof_list=np.arange(1,9)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,Y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,Y_train)
    score = model.score(X_test_rfe,Y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
#Apply Recursive Feature Selection on features selected using genetic algorithm
from sklearn.feature_selection import RFE
X=data[prediction_var_ga]
model = LinearRegression()
rfe = RFE(model, 8)
X_rfe = rfe.fit_transform(X,Y)  
model.fit(X_rfe,Y)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#Place selected features using hybrid GA+RFE approach in x
prediction_var_garfe=['SX','TCP','RER','EXIA','STDIE','SPE','NMV','THM']
x=data[prediction_var_garfe]


In [ ]:
#Apply scaling using standard saclar
from sklearn.preprocessing import StandardScaler
scaledfeatures=StandardScaler()
x=scaledfeatures.fit_transform(x)
y.value_counts()

In [ ]:
#Apply class balancing using SMOTE
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
x,y = smt.fit_sample(x,y)
y.value_counts()


In [ ]:
#Evaluate classifiers


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn import metrics

gnb=GaussianNB()
Accuracy=cross_val_score(gnb,x,y,cv=10).mean()
modelgnb=gnb.fit(x,y)
print ("Naive Bayes Accuracy=", Accuracy)

y_pred = cross_val_predict(modelgnb,x,y,cv=10)
confusion=metrics.confusion_matrix(y,y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print (confusion)
sensitivity = TP / float(FN + TP)

print("Naive Bayes sensitivity=",sensitivity)
specificity = TN / (TN + FP)
print("Naive Bayes specificity=",specificity)


Precision = TP / float(TP + FP)
Recall = TP / float(TP + FN)
F1 = 2*((Precision*Recall)/(Precision+Recall))
print ("Naive Bayes Precision=",Precision)
print ("Naive BayesRecall=", Recall)
print ("Naive Bayes FMeasure", F1)





sv=SVC(C=100,gamma=0.0001,kernel='rbf')
Accuracy=cross_val_score(sv, x, y,cv=10).mean()
print ("SVM Accuracy=", Accuracy)
y_pred = cross_val_predict(sv,x,y,cv=10)
confusion=metrics.confusion_matrix(y,y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print (confusion)
sensitivity = TP / float(FN + TP)

print("SVM sensitivity=",sensitivity)
specificity = TN / (TN + FP)
print("SVM specificity=",specificity)


Precision = TP / float(TP + FP)
Recall = TP / float(TP + FN)
F1 = 2*((Precision*Recall)/(Precision+Recall))
print ("SVM Precision=",Precision)
print ("SVM Recall=", Recall)
print ("SVM FMeasure", F1)


LR=LogisticRegression()

Accuracy=cross_val_score(LR,x,y,cv=10).mean()

print ("LR Accuracy=", Accuracy)


y_pred = cross_val_predict(LR,x,y,cv=10)
confusion=metrics.confusion_matrix(y,y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print (confusion)
sensitivity = TP / float(FN + TP)

print("LR sensitivity=",sensitivity)
specificity = TN / (TN + FP)
print("LR specificity=",specificity)

Precision = TP / float(TP + FP)
Recall = TP / float(TP + FN)
F1 = 2*((Precision*Recall)/(Precision+Recall))
print ("LR Precision=",Precision)
print ("LR Recall=", Recall)
print ("LR FMeasure", F1)






RF=RandomForestClassifier(n_estimators=100,random_state=1,bootstrap=True,criterion='entropy',max_depth=5,max_features=2,min_samples_leaf=9)
Accuracy=cross_val_score(RF,x,y,cv=10).mean()

print ("RF Accuracy=", Accuracy)
y_pred = cross_val_predict(RF,x,y,cv=10)
confusion=metrics.confusion_matrix(y,y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print (confusion)
sensitivity = TP / float(FN + TP)

print("RF sensitivity=",sensitivity)
specificity = TN / (TN + FP)
print("RF specificity=",specificity)


Precision = TP / float(TP + FP)
Recall = TP / float(TP + FN)
F1 = 2*((Precision*Recall)/(Precision+Recall))
print ("RF Precision=",Precision)
print ("RF Recall=", Recall)
print ("RF FMeasure", F1)

adaboost=AdaBoostClassifier(n_estimators=10, random_state=0,learning_rate=1)
Accuracy=cross_val_score(adaboost, x, y,cv=10).mean()
print ("Adaboost Accuracy=", Accuracy)
y_pred = cross_val_predict(adaboost,x,y,cv=10)
confusion=metrics.confusion_matrix(y,y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print (confusion)
sensitivity = TP / float(FN + TP)

print("Adaboost sensitivity=",sensitivity)
specificity = TN / (TN + FP)
print("Adaboost specificity=",specificity)


Precision = TP / float(TP + FP)
Recall = TP / float(TP + FN)
F1 = 2*((Precision*Recall)/(Precision+Recall))
print ("Adaboost Precision=",Precision)
print ("Adaboost Recall=", Recall)
print ("Adaboost FMeasure", F1)










In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [ ]:
#Define Parameters for RandomizedSearchCV() Method
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,50,100,150,200,300,400,500],
              'max_features':randint(1,9),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,10),
              }

In [ ]:
#Define RandomizedSearchCV() Method
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=9)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    return ht_params, ht_score

In [ ]:
#Hypertuning of Random Forest
rf_parameters, rf_ht_score = hypertuning_rscv(est, rf_p_dist, 40, x, y)

In [ ]:
#Parameters obtained using Random Forest
rf_parameters
 